In [1]:
from PYME.LMVis import VisGUI

%gui wx

Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.simulation
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.simulation
Trying to load 3rd party recipe module nep_fitting.recipe_modules.nep_fits
Loaded 3rd party recipe module nep_fitting.recipe_modules.nep_fits
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Trying to load 3rd party recipe module pyme_picasso.recipe_modules.conversion
Loaded 3rd party recipe module pyme_picasso.recipe_modules.conversion
Trying to load 3rd party recipe module pyme_4pi.recipe_modules.conversion
Loaded 3rd party recipe module pyme_4pi.recipe_modules.conversion
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Loaded 3rd part

In [2]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

DEBUG:PYME.LMVis.visCore:Creating VisGUI menu bar
DEBUG:PYME.LMVis.Extras:Initializing QPobjectSegment plugin
DEBUG:PYME.LMVis.Extras:Initializing animation plugin
DEBUG:PYME.LMVis.Extras:Initializing annotation plugin


quaternion module not found, disabling custom clip plane orientations
quaternion module not found, disabling custom clip plane orientations
New Canvas


c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:62: wxPyDeprecationWarning: Call to deprecated item. Use SetToolTip instead.
  self.bAdjustSnake.SetToolTipString('Adjust the parameters of fo the "snake" (active contour) used for curve locking')
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:73: wxPyDeprecationWarning: Call to deprecated item. Use InsertItem instead.
  self.lLabels.InsertStringItem(i, '%d' % i)
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:74: wxPyDeprecationWarning: Call to deprecated item. Use SetItem instead.
  self.lLabels.SetStringItem(i, 1, 'Structure %d' % i)
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:76: wxPyDeprecationWarning: Call to deprecated item. Use SetItem instead.
  self.lLabels.SetStringItem(0, 1, 'No label')
c:\users\laf62\code\python-microscopy\PYME\DSView\modules\annotation.py:95: wxPyDeprecationWarning: Call to deprecated item. Use SetToolTip ins

Saved Views are deprecated -  use the ones with animation instead and remove me!
Creating fold panel
GenPanels
Creating datasource panel
Image list size: 1
recipe.execute()
0
OpenGL - Version: b'4.6.0 NVIDIA 516.94'
Shader - Version: b'4.60 NVIDIA'
GL.GL_MAX_SAMPLES: 32, GL.GL_SAMPLES: 4


DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.DefaultShaderProgram'>


Ev Idle
('points', 't')
refreshed
Input datasources:  []
Creating layers GUI
height:  0
NB best size: wx.Size(0, 0)


In [3]:
import numpy as np
from PYME.IO import tabular

# PARAMETERS to Change

# Save location
savedir = "F:\\Rtn4_Paper_Data\\4Pi_data\\Cross-sectional_angles_analysis"
ver = 'V7-vector-rejection' # analysis version to not overwrite old analyses
ball_r = 50 #radius used for query_ball_points
max_nn = 200 #1000 #maximum # of nearest neighbors before collapsing them into one point
min_dist = 10 #minimum distance between skeleton vertices in determining the vector that points from each vertex
              #to the closest vertex that's at least this distance away (helps make nice backbone vectors)
sim_p = 1 # number of simulated points per skeleton point
high_filt=200          # Threshold for max distance a Rtn4 point can be from mesh and still be included in analysis
low_filt=-100          # Threshold for min distance a Rtn4 point can be from mesh and still be included in analysis
distance_filter = 2000  # Threshold for maximum distance a point can be from the skeleton to be in the angles calculation
low_perp = 89          # Thresholdf or min angle a point can be relative to skeleton
high_perp = 91         # Threshold of max angle a point can be relative to the skeleton

#making save-folder if it doesn't exist (DON'T TOUCH THE NEXT 5 LINES)
from pathlib import Path
import os
p = Path(savedir)
if not p.exists():
    os.mkdir(savedir)
workdir = os.getcwd()
new_path = savedir + '\\analysis_results_' + ver
p1 = Path(new_path)
if not p1.exists():
    os.mkdir(new_path)

#######################################################################################################################
# Rtn4 parameters and files (for multiple ROIs)
#######################################################################################################################

# pass pointcloud and mesh files here for Rtn

# Below are the parameters needed for analyzing multiple Rtn4 ROIs together. You will need to name each file as:
# points: 'Cellxx_ROIy.hdf'
# meshes: 'Cellxx_ROIy_mesh.stl'
# Where xx is the number of the cell from which the ROI comes from in that dataset
# Where y is the number of the ROI. It doesn't matter how many ROIs or what number they start at as long as you specify those
# two parameters below AND the ROI numbers increase by +1 continuously.


rtn_num = 3#10           # How many ROIs of Rtn4 do you want to analyze?
rtn_roi_start = 1#7#          # What number does the first ROI start at?
foldname = "F:\\Rtn4_Paper_Data\\4Pi_data\\Cross-sectional_angles_analysis\\Rtn4_Data"
filename = '\\Rtn4_ROI'
rtn_chan = 'chan1'    # channel name for Rtn4 data
rtn_save = 'rtn4'
rtn_list = []
rtn_mesh_list = []
rtn_skel_list = []
for a in range(rtn_roi_start, rtn_num+rtn_roi_start):
    rtn_list.append(foldname+filename+str(int(a))+'.hdf')
    rtn_mesh_list.append(foldname+filename+str(int(a))+'_shrinkwrap.stl')
    rtn_skel_list.append(foldname+filename+str(int(a))+'_skeleton.stl')

# make dictionary with variables
rtn_param = {"chan": rtn_chan, "save": rtn_save, "low_filt": low_filt, "high_filt": high_filt,
            "distance_filter": distance_filter, "roi_start": rtn_roi_start, "low_perp": low_perp,
            "high_perp": high_perp}

#######################################################################################################################
# Sec61B parameters and files
#######################################################################################################################

sec_num = 3           # How many ROIs of Sec61B do you want to analyze?
sec_roi_start = 1          # What number does the first ROI start at?
sec_foldname = "F:\\Rtn4_Paper_Data\\4Pi_data\\Cross-sectional_angles_analysis\\Sec61B_Data"
sec_filename = '\\Sec61B_ROI'
sec_chan = 'chan0'     # channel name for antibody data
sec_save = 'sec61B'
sec_list = []
sec_mesh_list = []
sec_skel_list = []
for c in range(sec_roi_start, sec_num+sec_roi_start):
    sec_list.append(sec_foldname+sec_filename+str(int(c))+'.hdf')
    sec_mesh_list.append(sec_foldname+sec_filename+str(int(c))+'_shrinkwrap.stl')
    sec_skel_list.append(sec_foldname+sec_filename+str(int(c))+'_skeleton.stl')

# make dictionary with variables
sec_param = {"chan": sec_chan, "save": sec_save, "low_filt": low_filt, "high_filt": high_filt,
            "distance_filter": distance_filter, "roi_start": sec_roi_start, "low_perp": low_perp,
            "high_perp": high_perp}

#######################################################################################################################
# KDEL parameters and files
#######################################################################################################################

kdel_num = 3           # How many ROIs of KDEL do you want to analyze?
kdel_roi_start = 1          # What number does the first ROI start at?
kdel_foldname = "F:\\Rtn4_Paper_Data\\4Pi_data\\Cross-sectional_angles_analysis\\KDEL_Data"
kdel_filename = '\\KDEL_ROI'
kdel_chan = 'chan0'     # channel name for Antibody data
kdel_save = 'kdel'
kdel_list = []
kdel_mesh_list = []
kdel_skel_list = []
for c in range(kdel_roi_start, kdel_num+kdel_roi_start):
    kdel_list.append(kdel_foldname+kdel_filename+str(int(c))+'.hdf')
    kdel_mesh_list.append(kdel_foldname+kdel_filename+str(int(c))+'_shrinkwrap.stl')
    kdel_skel_list.append(kdel_foldname+kdel_filename+str(int(c))+'_skeleton.stl')

# make dictionary with variables
kdel_param = {"chan": kdel_chan, "save": kdel_save, "low_filt": low_filt, "high_filt": high_filt,
            "distance_filter": distance_filter, "roi_start": kdel_roi_start, "low_perp": low_perp,
            "high_perp": high_perp}
    

In [4]:
import numpy as np
from PYME.IO import tabular

# generate 3D Gaussian
# points = np.random.randn(100,3)*100

#######################################################################################################################
# Define needed functions
#######################################################################################################################

def add_ds_from_Nx3(points, pipeline, pymevis, ds_name='points', color=None, normals=None):
    """
    Quickly add points/normals to the pipeline and pymevis display.
    """
    
    d = {'x': points[:,0], 'y': points[:,1], 'z': points[:,2]}
    
    if color is not None:
        d['c'] = color
        
    if normals is not None:
        d['xn'] = normals[:,0]
        d['yn'] = normals[:,1]
        d['zn'] = normals[:,2]

    # create tabular mappingFilter data source and add it to pymevis
    pipeline.addDataSource(ds_name, tabular.mappingFilter(d))

    # select this data source (optional, but helps support "default behavior")
    pipeline.selectDataSource(ds_name)

    # Add a pointcloud layer that displays data source named 'points' (the one we just added)
    pymevis.add_pointcloud_layer(ds_name=ds_name)
    
    if normals is not None:
        pymevis.glCanvas.layers[-1].display_normals=True
        pymevis.glCanvas.layers[-1].normal_scaling=25.0

def save_snapshot(canvas, file_name=None):
    if True:
        pixel_size=None
        
        if file_name is None:
            file_name = wx.FileSelector('Save current view as', wildcard="PNG files(*.png)|*.png",
                            flags=wx.FD_SAVE | wx.FD_OVERWRITE_PROMPT)
        
        if file_name:
            snap = canvas.getIm(pixel_size, GL_RGB)
            print(snap.dtype, snap.shape, snap.max())
            if snap.ndim == 3:
                img = PIL.Image.fromarray(snap.transpose(1, 0, 2))
                #img = toimage(snap.transpose(1, 0, 2))
            else:
                img = PIL.Image.fromarray(snap.transpose())
                #img = toimage(snap.transpose())
            
            img = img.transpose(PIL.Image.FLIP_TOP_BOTTOM)
            
            if not file_name.endswith('.png'):
                img.save('{}.png'.format(file_name))
            else:
                img.save('{}'.format(file_name))
                
def interior_points_from_sdf(sdf, r_max=1, centre=(0,0,0), dx_min=1, p=0.1, eps=0):
    '''
    Generate points from a signed distance function. Effectively does octree-like subdivision of the function domain to
    assign points on a regular grid, then passes through a Monte-Carlo acceptance function to simulate labelling efficiency
    
    Parameters
    ----------
    sdf : function
        the signed distance function. Should be of the form dist = sdf(pts) where pts is a 3xN ndarray/
    r_max: float
        The maximum radius of the object (from centre)
    centre : 3-tuple / array of float
        The centre of the object
    dx_min : float
        The target side length of a voxel. Effectively a density parameter (density = 1/dx_min^3).
    p : float
        Monte-Carlo acceptance probability.

    Returns
    -------
    
    verts : 3xN ndarray of fluorophore positions

    '''
    dx = 1.2 * r_max
    
    vx, vy, vz = [v.ravel() for v in np.mgrid[-1:2, -1:2, -1:2]]
    
    verts = dx * np.vstack([vx, vy, vz]) + np.array(centre)[:,None]
#     print(verts.shape)
    
    c_offs = np.array(
        [[-1, -1, -1], [1, -1, -1], [1, 1, -1], [-1, 1, -1], [-1, -1, 1], [1, -1, 1], [1, 1, 1], [-1, 1, 1]])
    #test_offs =
    
    while dx > dx_min:
        dx /= 2.0
        #test and discard.
        corners = [verts + dx * c_offs[i][:, None] for i in range(8)]
        corner_dists = [sdf(c) for c in corners] + [sdf(verts), ]
#         print(corner_dists)
        
        #corner_dists = [sdf(verts),]
        
        #contains_points = np.abs(np.sum([np.sign(c) for c in corner_dists], axis=0)) < 9
        # for some reason the sign test doesn't seem to work properly. Use a generous
        # distance based test instead
        contains_points = np.min([c for c in corner_dists], axis=0) < 2 * dx
#         print(dx)
#         print(corner_dists)
        
        verts = verts[:, contains_points]
#         print(verts.shape)
        
        #subdivide
        verts = np.concatenate([verts + dx * c_offs[i][:, None] for i in range(8)], axis=1)
        #print(verts.shape)
    
    # because we use a fairly relaxed / conservative criterea for discarding above (which will
    # effectively match the cell containing the surface AND it's neighbours)
    # we perform a more stringent test on the final set of points.
    corners = [verts + dx * c_offs[i][:, None] for i in range(8)]
    corner_dists = [sdf(c) for c in corners]
    
    sv = sdf(verts)
    contains_points = ((np.sum([np.sign(c) for c in corner_dists], axis=0) == -8) | (sv < dx)) & (sv < eps)
    verts = verts[:, contains_points]
    
    #print(verts.shape)
    
    return verts[:, np.random.rand(verts.shape[1]) < p]

def grad_sdf(pts, sdf, delta=0.1):
    """
    Gradient of the signed distance function, calculated via central differences.
    
    Parameters
    ----------
    pts : np.array
        3 x N array of points on which we evaluate the gradient
    sdf : function
        Signed-distance function. Expects 3xN vector of x, y, z coordinates.
    delta : float
        Shift in gradient direction.
    """
    d2 = delta/2.0
    hx = np.array([d2,0,0])[:,None]
    hy = np.array([0,d2,0])[:,None]
    hz = np.array([0,0,d2])[:,None]
    dx = (sdf(pts + hx) - sdf(pts - hx))/delta
    dy = (sdf(pts + hy) - sdf(pts - hy))/delta
    dz = (sdf(pts + hz) - sdf(pts - hz))/delta
    
    return dx, dy, dz

###########################################################################################################
# New skeletonization function
###########################################################################################################
from ch_shrinkwrap.recipe_modules.surface_feature_extraction import SkeletonizeMembrane

# function definition to compute magnitude of a vector
import numpy
import math
def magnitude(vector):
    return math.sqrt(sum(pow(element, 2) for element in vector))

In [ ]:
### Testing new vector from skeleton to point calculation
import numpy as np
sp = np.array([0,0,0])
sp2 = np.array([1,0,0])
b = sp2 - sp
b_hat = b/np.linalg.norm(b)
a = np.array([1,1,0])
avec = a-sp
# Vector rejection: a2 = a - bhat x (a dot b / |b|)
ap = avec-b_hat*(np.dot(avec,b)/np.linalg.norm(b))
ap

In [ ]:
# STill testing
import numpy as np
z = np.arange(300).reshape(100,3)
np.linalg.norm(z[1])

In [5]:
import os
from os.path import exists
import time
import PIL
import wx
from OpenGL.GL import GL_LUMINANCE, GL_RGB
from pathlib import Path
import pickle

def point_angles_V3(file_list, mesh_list, skel_list, param_list, sim_p):
    """
    calculate the angle between vector1 (pointing from each data point to the closest skeleton point) and vector2
    (e0 vector of each skeleton point that should point along axis of tubule) relative to the z-direction of each
    tubule)
    
    update on 8-5-2022: I have changed the analysis from using the e0 vector. These seemed too inconstistent in
    their directions, and I was worried they'd lead to artifacts in the analysis. Now, I reassign each skeleton 
    vertex to a coordinate that is the mean coordinate of that vertex and all the vertices on the skeleton within some
    radius (20 nm seems to work well). I then make a vector that points from each vertex on the skeleton to the
    closest vertex on the skeleton that is at least x distance away (5-10 nm works well). 
    
    update on 11-11-2022: I am moving away from many small ROIs to a few large ROIs. Previously, since creating
    skeletons within the script loop didn't take too long, I opted for this instead of manually creating skeletons
    for each ROI. Now, it takes a few hours to create a skeleton for one of these larger ROIs, so I have opted to
    manually create the skeletons beforehand and load them into the analysis. Thus, I am updating this function
    to load in the skeletons instead of creating them each time.
    
    update on 12/14/2022: I am changing the way the vectors are calculated that point from the skeleton to the 
    localizations. Previously, the vectors pointed from the neareast skeleton point to each localization. 
    Now, the vector rejection is calculated instead. This ensures that the vector is actually perfectly perpendicular
    to the skeleton. It should also allow me to remove the filtering of points that are not close to pependicular to
    the skeleton. See vector projection wiki for info about vector rejection: https://en.wikipedia.org/wiki/Vector_projection.
    The formula is a2 = a - bhat (a dot b / |b|), where a is the vector pointing from the skeleton to the localization, 
    a2 is the vector rejection, b is the skeleton axis, bhat is the unit vector of b, |b| is the magnitude of b.
    """
    # initiate variables
    angles_all = []
    angles_z_all = []
    sim_angles_all = []
    sim_angles_z_all = []
    skel_dist_all = []
    skel_ind_all = []
    sim_dist_all = []
    sim_ind_all = []
    points_all = []
    skeleton_all = []
    e0_all = []
    e0_hat_all = []
    e1_all = []
    vecs_hat_all = []
    vecs_hat_ind_all = []
    v_hat_all = []
    simv_hat_all = []
    vecs_hat_sim_all = []
    sim_points_all = []
    sim_v_all = []
    ps_deg_all = []
    v_norm_all = []
    save_screenshots = new_path + '\\screenshots'
    p3 = Path(save_screenshots)
    if not p3.exists():
        os.mkdir(save_screenshots)

    for d in range(len(file_list)):
        angles = []
        angles_z = []
        sim_angles = []
        sim_angles_z = []
        start_time = time.time()

        # Load and display points
        print('start')
        pymevis.OpenFile(file_list[d])

        # define mesh sdf
        from PYME.experimental.isosurface import distance_to_mesh

        mesh_sdf = lambda pts: distance_to_mesh(pts.T, mesh)

        print('loading mesh')
        # Load and display mesh
        from PYME.experimental._triangle_mesh import TriangleMesh
        from PYME.LMVis.layers.mesh import TriangleRenderLayer

        mesh = TriangleMesh.from_stl(mesh_list[d])

        mesh_name = pipeline.new_ds_name('surf')
        pipeline.recipe.namespace[mesh_name] = mesh
        layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = 'SolidCyan')
        pymevis.add_layer(layer)

        # grab the bounding box
        mesh_bbox = pymevis.glCanvas.layers[-1]._bbox
        
        pipeline.selectDataSource('surf0')
        sw = pipeline.selectedDataSource
        sw_surf = sw._vertices['position'][sw._vertices['halfedge']!=-1]
      
        # Load and display skeleton (this is replacing the skeleton creation within the script approach because
        # it takes so long to make the skeleton. Better to do it once and then load it in for analyses

        from PYME.experimental._triangle_mesh import TriangleMesh
        from PYME.LMVis.layers.mesh import TriangleRenderLayer

        mesh = TriangleMesh.from_stl(skel_list[d])

        mesh_name = pipeline.new_ds_name('skeleton')
        pipeline.recipe.namespace[mesh_name] = mesh
        layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = 'SolidMagenta')
        pymevis.add_layer(layer)

        # grab the bounding box
        mesh_bbox = pymevis.glCanvas.layers[-1]._bbox
        
        # assign values to permanent lists
        pipeline.selectDataSource('skeleton0')
        skelly = pipeline.selectedDataSource
        orig_skeleton = skelly._vertices['position'][skelly._vertices['halfedge']!=-1]
       
        from scipy.spatial import cKDTree
        tree_orig = cKDTree(orig_skeleton)

        # Move each vertex to the average position of all of its neighbors within a 'ball_r' radius
        # Also trim out vertices that have a # of nearest neighbors > than max_nn (these are the q-tip looking spots)
        # This doesn't make the problem areas better, it just cuts them out entirely
        from collections import OrderedDict
        skeleton = orig_skeleton
        print(skeleton)
        print('search here')
        bad_inds = []
        a_ind = tree_orig.query_ball_point(skeleton, ball_r)
        for trim in range(len(a_ind)):
            skeleton[trim] = np.mean(skeleton[a_ind[trim]],axis=0)
            if len(a_ind[trim]) > max_nn:
                bad_inds.append(a_ind[trim])
        flat_bad_inds = [element for sublist in bad_inds for element in sublist]
        flat_unique = list(OrderedDict.fromkeys(flat_bad_inds))
        skeleton = np.delete(skeleton, flat_unique, axis=0)
        # Make KDTree based on the mean skeleton
        mean_tree_skeleton = cKDTree(skeleton)

        # Find the closest neighbor that is at least 'min_dist' away from point being queried.
        nneigh_ind = np.zeros([len(skeleton),1])
        nneigh_dist = np.zeros([len(skeleton),1])
        for aa in range(len(skeleton)):
            dist, ind = mean_tree_skeleton.query(skeleton[aa], k=len(skeleton), distance_upper_bound=min_dist*2)
            for ab in range(len(dist)):
                if dist[ab] > min_dist:
                    nneigh_ind[aa] = ind[ab]
                    nneigh_dist[aa] = dist[ab]
                    break

        # Calculate vectors pointing from each point to its nearest neighbor that is at least 'min_dist' away.
        skel_vecs = np.zeros([len(skeleton),3])
        for ac in range(len(skeleton)):
            if nneigh_ind[ac] >= len(skeleton):
                continue
            else:
                vec = skeleton[ac] - skeleton[int(nneigh_ind[ac][0])]
                skel_vecs[int(ac)] = vec

        # Normalize
        skel_dist_mag = np.c_[nneigh_dist.ravel(),nneigh_dist.ravel(),nneigh_dist.ravel()]
        vecs_hat = skel_vecs/skel_dist_mag
        vecs_hat_all.append(vecs_hat)

        # Add those vectors to the skeleton in the GUI
        add_ds_from_Nx3(skeleton, pipeline, pymevis, ds_name=f'nn-mean-skel-{min_dist}nm', normals=vecs_hat)
        
        # store points/vectors
        skeleton_all.append(skeleton)

        print('starting filters')
        # Import ExtractTableChannel Recipe
        from PYME.recipes.localisations import ExtractTableChannel
        recipe = pipeline.recipe
        # Extract  points as 'points' DataSource
        recipe.add_modules_and_execute([ExtractTableChannel(recipe, inputName = 'filtered_localizations', 
                                                            outputName = 'points', channel = param_list['chan'])])
        
        # Select DataSource
        pipeline.selectDataSource('points')
        
        # Calculate distance from points to surface
        from PYME.recipes.surface_fitting import DistanceToMesh
        recipe = pipeline.recipe
        recipe.add_modules_and_execute([DistanceToMesh(recipe, input_mesh = 'surf0', input_points = 'points', 
                                                       output = 'points_filt')])
        # Select datasource that now has the distance_to_surf0 values associated with it
        pipeline.selectDataSource('points_filt')

        # Dictionary with info about what filter to use and what values
        filt_dict = {'distance_to_surf0': [param_list['low_filt'],param_list['high_filt']]}

        # Use PYME FilterTable to filter by distance to mesh (SDF) using low_filt and high_filt variables
        from PYME.recipes.tablefilters import FilterTable
        recipe.add_modules_and_execute([FilterTable(recipe, inputName = 'points_filt', filters = filt_dict, 
                                                    outputName = 'sdf_filtered_points')])

        # Select the points that passed the SDF filter above
        pipeline.selectDataSource('sdf_filtered_points')

        # Extract x,y,z coordinates for simulated points
        x = pipeline['x']
        y = pipeline['y']
        z = pipeline['z']

        # Merge x,y,z coordinates into one variable 'points'
        points = np.c_[x.ravel(),y.ravel(),z.ravel()]
        print(points)
        # skel_dist = distance from each point to closest skeleton point; skel_ind = index of the closest point in 'skeleton'
        print('starting skeleton query')
        query_time = time.time()
        skel_dist, skel_ind = mean_tree_skeleton.query(points, k=1)
        print('search here')
        print(skel_ind)
        # Create boolean Filter to filter out points that are too far from the skeleton
        bool_dist = skel_dist <= param_list['distance_filter']
        print(skel_dist)
        print("skeleton query ", d," took", time.time() - query_time, "to run")
        # Filter out points that are too far from the skeleton
        points = points[bool_dist]
        skel_dist = skel_dist[bool_dist]
        skel_ind = skel_ind[bool_dist]


####### This part starts the calculations of angles
        print('starting angle calculations')
        angle_time = time.time()
        # vector from 1st nearest neighbor to point

        # Calculate vector pointing from points to skeleton
        vp = points-skeleton[skel_ind]

        # Calculate vector rejections
        vecs_hat_ind = vecs_hat[skel_ind] #only relevant skeleton vectors

        v = []
        v_hat = []
        for vrej in range(len(vp)):
            v_temp = vp[vrej]-vecs_hat_ind[vrej]*(np.dot(vp[vrej],vecs_hat_ind[vrej])/np.linalg.norm(vecs_hat_ind[vrej]))
            v.append(v_temp)
            v_hat_temp = v_temp/np.linalg.norm(v_temp)
            v_hat.append(v_hat_temp)
        v = np.asarray(v)
        v_hat = np.asarray(v_hat)

        # make skel_dist an [n, 3] array to match v
        skel_dist_mag = np.c_[skel_dist.ravel(),skel_dist.ravel(),skel_dist.ravel()]

        # normalized/unit vector pointing from skeleton to point (divide each part of the vector by magnitidue of vector)
        # v_hat = v/skel_dist_mag

        
        # Adding new filter to exclude points at angles not close to
        # Perpendicular to the skeleton axis (vecs_hat)
        ps_angle = []
        for pa in range(len(v_hat)):
            angle = np.arccos(np.dot(v_hat[pa],vecs_hat_ind[pa])/(magnitude(v_hat[pa])*magnitude(vecs_hat_ind[pa])))
            ps_angle.append(angle)
        ps_angle = np.asarray(ps_angle)
        from numpy import pi
        ps_deg = ps_angle * (180/pi)
        # ps_low = ps_deg > param_list['low_perp'] # filter out angles below 85 degrees
        # ps_high = ps_deg < param_list['high_perp'] # filter out angles above 95 degrees
        # ps_filt = ps_low * ps_high # combine filters
        # # Apply filters to everything
        # ps_deg = ps_deg[ps_filt]
        # points = points[ps_filt]
        # skel_dist = skel_dist[ps_filt]
        # skel_ind = skel_ind[ps_filt]
        # v_hat = v_hat[ps_filt]
        # vecs_hat_ind = vecs_hat_ind[ps_filt]
        
####### UNDER CONSTRUCTION: Normalize point vectors by the size of surface nearby
        # Create KDTree from shrink-wrapped surface
        surf_tree = cKDTree(sw_surf)
        used_skel = skeleton[skel_ind]
        # Find nearest neighbors of surface vertices to used skeleton points
        sw_dist, sw_ind = surf_tree.query(used_skel, k=1)
        # Calculate vector pointing from surface vertices to closest point on skeleton
        sw_vec = sw_surf[sw_ind]-used_skel
        # Determine relevant skeleton vectors
        #sw_skel_vecs = vecs_hat[sw_ind]
        # Calculate vector rejections
        v_sw = []
        v_sw_hat = []
        for vrej2 in range(len(sw_vec)):
            v_sw_temp = sw_vec[vrej2]-vecs_hat_ind[vrej2]*(np.dot(sw_vec[vrej2],vecs_hat_ind[vrej2])/np.linalg.norm(vecs_hat_ind[vrej2]))
            v_sw.append(v_sw_temp)
            v_sw_hat_temp = v_sw_temp/np.linalg.norm(v_sw_temp)
            v_sw_hat.append(v_sw_hat_temp)
        v_sw = np.asarray(v_sw)
        v_sw_hat = np.asarray(v_sw_hat)
        
        # Normalize distance to localizations by the distance from the skeleton to closest surface vertex
        #print(v)
        v_mag = np.linalg.norm(v,axis=1)
        mag_dist_filt = v_mag < param_list['distance_filter']
        v_mag = v_mag[mag_dist_filt]
        surf_mag = np.linalg.norm(v_sw,axis=1)
        surf_mag = surf_mag[mag_dist_filt]
        v_norm = v_mag/surf_mag
        v_norm_all.append(v_norm)
        # Apply filters to everything
        ps_deg = ps_deg[mag_dist_filt]
        points = points[mag_dist_filt]
        skel_dist = skel_dist[mag_dist_filt]
        skel_ind = skel_ind[mag_dist_filt]
        v_hat = v_hat[mag_dist_filt]
        vecs_hat_ind = vecs_hat_ind[mag_dist_filt]
    
        # Append distances and inds to permanent array
        ps_deg_all.append(ps_deg)
        points_all.append(points)
        skel_dist_all.append(skel_dist)
        skel_ind_all.append(skel_ind)
        v_hat_all.append(v_hat)
        vecs_hat_ind_all.append(vecs_hat_ind)
    
    
        # now let's construct the frame of reference

        # Z-direction unit vector
        zdir = np.array([0,0,1])

        # pick dir along skeleton axis as first vector in frame
        frame0 = vecs_hat[skel_ind] #frame0 = e0_hat[skel_ind]

        # component of z not in the direction of e0 or portion of Z-dir that is orthogonal to frame0
        frame1 = zdir[None,:] - frame0*(frame0*zdir[None,:]).sum(1)[:,None]

        # orthogonal to other two vectors in the frame
        frame2 = np.cross(frame0,frame1,axis=1)

        # make them unit vectors (vector divided its magnitude)
        frame1 = frame1/np.linalg.norm(frame1,axis=1)[:,None]
        frame2 = frame2/np.linalg.norm(frame2,axis=1)[:,None]

        # Determine angles between relative z-direction of tubule and vector between point and skeleton
        #angles.append(np.arccos((v_hat*frame1).sum(1)))  # this works because frame1 is also normalized (unit length=1)
        angle_list = []
        angleb_list = []
        for pb in range(len(v_hat)):
            angle = np.arccos(np.dot(v_hat[pb],frame1[pb])/(magnitude(v_hat[pb])*magnitude(frame1[pb])))
            angle_list.append(angle)
            angleb = np.arccos(np.dot(v_hat[pb],zdir)/(magnitude(v_hat[pb])*magnitude(zdir)))
            angleb_list.append(angleb)
        angles_all.append(angle_list)
        angles_z_all.append(angleb_list)
        #angles.append(np.arccos(np.dot(v_hat,frame1)/(magnitude(v_hat)*magnitude(frame1))))
        # Determine angles between true z-direction and vector between point and skeleton
        #angles_z.append(np.arccos((v_hat*zdir).sum(1)))
        #angles_z.append(np.arccos(np.dot(v_hat,zdir)/(magnitude(v_hat)*magnitude(zdir))))
        
#####   Simulation zone
        
        rng = np.random.default_rng(12345)
        # equations for making the vectors
        r = 1 # radius of points sampled in the circle on the plane

        f_all = []
        for bd in range(len(skeleton)):
            p1 = skeleton[bd]
            n = vecs_hat[bd] # vector normal to the plane
            nabs = np.absolute(n)
            indices = (-n).argsort()[:3]
            v = np.zeros(3)
            v[indices[1]] = -n[indices[0]]
            v[indices[0]] = n[indices[1]]
            v[indices[2]] = 0
            u = np.cross(n,v)
            #normalize vectors
            v_norm = v/np.linalg.norm(v)
            u_norm = u/np.linalg.norm(u)
            f = []
            for be in range(sim_p):
                rho = rng.random()*(2*np.pi)
                circ_p = p1 + r * v_norm * np.cos(rho) + r * u_norm * np.sin(rho)
                circ_vec = p1-circ_p
                f.append(skeleton[bd] + np.random.randint(0,100)*(circ_vec/np.linalg.norm(circ_vec)))
            f = np.asarray(f)
            f_all.append(f)
        f_all = np.asarray(f_all)
        f_flat = [element for sublist in f_all for element in sublist]
        f_flat = np.asarray(f_flat)
        sim_points_all.append(f_flat)
        

        # for some reason, f_flat sometimes has NaN values which result in a sim_ind = len(skeleton)
        # these next lines filter these out
        sim_dist, sim_ind = mean_tree_skeleton.query(f_flat, k=1)
        
        bad_sim = sim_ind == len(skeleton)
        f_flat = np.delete(f_flat, bad_sim, axis=0)
        sim_ind = np.delete(sim_ind, bad_sim, axis=0)
        sim_dist = np.delete(sim_dist, bad_sim, axis=0)
        print(f_flat)
        add_ds_from_Nx3(f_flat, pipeline, pymevis, ds_name='simulated-points', normals=None)
        
        # Calculate angles to simulated points
        sim_v = f_flat - skeleton[sim_ind]           #Vector pointing from each simulated point to nearest skeleton point
        # make skel_dist an [n, 3] array to match sim_v
        sim_dist_mag = np.c_[sim_dist.ravel(),sim_dist.ravel(),sim_dist.ravel()]
        # normalized/unit vector pointing from skeleton to point (divide each part of the vector by magnitidue of vector)
        simv_hat = sim_v/sim_dist_mag
        simv_hat_all.append(simv_hat)
        # pick dir along skeleton axis as first vector in frame
        sim_frame0 = vecs_hat[sim_ind]
        # component of z not in the direction of e0 or portion of Z-dir that is orthogonal to frame0
        sim_frame1 = zdir[None,:] - sim_frame0*(sim_frame0*zdir[None,:]).sum(1)[:,None]
        # orthogonal to other two vectors in the frame
        sim_frame2 = np.cross(sim_frame0,sim_frame1,axis=1)
        # make them unit vectors (vector divided its magnitude)
        sim_frame1 = sim_frame1/np.linalg.norm(sim_frame1,axis=1)[:,None]
        sim_frame2 = sim_frame2/np.linalg.norm(sim_frame2,axis=1)[:,None]
        # Determine angles between relative z-direction of tubule and vector between point and skeleton
        # sim_angles.append(np.arccos((simv_hat*sim_frame1).sum(1)))  # this works because frame1 is also normalized (unit length=1)
        sangle_list = []
        sangleb_list = []
        for pc in range(len(simv_hat)):
            sangle = np.arccos(np.dot(simv_hat[pc],sim_frame1[pc])/(magnitude(simv_hat[pc])*magnitude(sim_frame1[pc])))
            sangle_list.append(sangle)
            sangleb = np.arccos(np.dot(simv_hat[pc],zdir)/(magnitude(simv_hat[pc])*magnitude(zdir)))
            sangleb_list.append(sangleb)
        sim_angles_all.append(sangle_list)
        sim_angles_z_all.append(sangleb_list)
        # sim_angles.append(np.arccos(np.dot(simv_hat,sim_frame1)/(magnitude(simv_hat)*magnitude(sim_frame1))))
                        
        # Determine angles between true z-direction and vector between point and skeleton
        # sim_angles_z.append(np.arccos((simv_hat*zdir).sum(1)))
        # sim_angles_z.append(np.arccos(np.dot(simv_hat,zdir)/(magnitude(simv_hat)*magnitude(zdir))))
        sim_dist_all.append(sim_dist)
        sim_ind_all.append(sim_ind)
        vecs_hat_sim = vecs_hat[sim_ind]
        vecs_hat_sim_all.append(vecs_hat_sim)
        sim_v_all.append(sim_v)
#####   End of simulation zone


        print("angle calculations ", d," took", time.time() - angle_time, "to run")
        # Edit layers in Visgui to prepare for screenshot

        pymevis.glCanvas.layers[0].visible=True
        pymevis.glCanvas.layers[1].visible=False
        pymevis.glCanvas.layers[2].visible=True
        pymevis.glCanvas.layers[2].point_size = 5       # Change filtered data point size to 15 nm
        #pymevis.glCanvas.layers[-2].visible=False
        pymevis.glCanvas.layers[-1].visible=False

        
        #    pipeline.selectDataSource('skeleton')            # Select skeleton
        #    pymevis.glCanvas.layers[4].cmap = 'R'            # Change skeleton color to red
        #    pipeline.selectDataSource('sdf_filtered_Rtn4')   # Select filtered Rtn4 data
        #    pymevis.glCanvas.layers[2].cmap = 'G'            # Change filtered Rtn4 data to green

        import PIL
        import wx
        from OpenGL.GL import GL_LUMINANCE, GL_RGB
        new_savedir = save_screenshots + '\\' + param_list['save'] + 'screenshot_ROI'+str(int(param_list['roi_start']+d))+'.png'         # Create path + file name variable
        pymevis.OnResetRotation()
        pymevis.SetFit()
        pymevis.glCanvas.view.translation[1] += -100
        pymevis.glCanvas.scaleBarLength = 100
        pymevis.glCanvas.view.scale *= 0.8
        # pymevis.glCanvas.layers[6].visible=True
        # pymevis.glCanvas.layers[6].point_size=15
        # pymevis.glCanvas.layers[6].method='spheres'
        pymevis.glCanvas.Refresh()
        save_snapshot(pymevis.glCanvas, new_savedir)     # Take screenshot of current Canvas and save at the path + file name provided by sim_savedir

        print("Loop #", d,"took", time.time() - start_time, "to run")
    angles_all = np.asarray(angles_all)
    angles_z_all = np.asarray(angles_z_all)
    sim_angles_all = np.asarray(sim_angles_all)
    sim_angles_z_all = np.asarray(sim_angles_z_all)


    output = {'angles': angles_all, 'angles_z': angles_z_all, 'sim_angles': sim_angles_all, 'sim_angles_z': sim_angles_z_all,
              'skel_dists': skel_dist_all, 'skel_inds': skel_ind_all, 'sim_dists': sim_dist_all,
              'sim_inds': sim_ind_all, 'points': points_all, 'skeleton': skeleton_all, 
              'sim_points': sim_points_all, 'v_hat': v_hat_all, 'sim_v': sim_v_all, 
              'tube_axis': vecs_hat_all, 'ps_angle': ps_deg_all, 'vecs_hat_ind': vecs_hat_ind_all,
             'vecs_hat_sim': vecs_hat_sim_all, 'Norm_dists': v_norm_all}
    
    save_output_path = new_path + '\\point_angles_output_dicts'
    p2 = Path(save_output_path)
    if not p2.exists():
        os.mkdir(save_output_path)
    save_output_file = save_output_path + '\\' + param_list['save'] + '_point_angles_output_dicts_ROI' + file_list[0][-5:-4] + '-' + str(int(file_list[0][-5:-4])+len(file_list)-1) + '.npy' #'_point_angles_output.npy' 
    np.save(save_output_file, output, allow_pickle=True)
    
    return output #angles, skel_dist_all

In [6]:
############################## Rtn4 Analysis ##############################
rtn_output = point_angles_V3(file_list=rtn_list, mesh_list=rtn_mesh_list, skel_list=rtn_skel_list, 
                             param_list=rtn_param, sim_p = sim_p) 

start
Creating Pipeline


DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.localisations.Pipelineify object at 0x0000015305CC8CA8>
DEBUG:PYME.recipes.localisations:no events found
DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.localisations.ProcessColour object at 0x0000015377268E28>
DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.tablefilters.FilterTable object at 0x00000153020D2468>
DEBUG:PYME.recipes.recipe:Module timings: {'localisations.AddPipelineDerivedVars': 0.0, 'localisations.ProcessColour': 0.017951488494873047, 'tablefilters.FilterTable': 0.02994847297668457}


recipe.execute()
Pipeline Created


DEBUG:PYME.LMVis.VisGUI:Creating tabs
DEBUG:PYME.LMVis.VisGUI:Finished creating tabs


0
Gui stuff done
loading mesh
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (114004, 3), faces.shape = (229518, 3)
iterating edges
done initializing halfedges
mesh_layer update
mesh_layer update


...

pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
mesh_layer update
mesh_layer update
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
[[10927.528    9945.874    -185.70287]
 [11604.974   13650.515    -261.0359 ]
 [12222.559    9428.673    -263.4084 ]
 ...
 [19490.355    5152.7544   -290.5044 ]
 [11707.271    7462.7393   -227.48262]
 [16431.594    5003.765    -166.73659]]
starting skeleton query
search here
[ 14966  26978  37287 ... 157647  29421 112668]
[ 64.80270214  37.46893304  86.08052315 ... 117.60676368 105.86726905
  46.50351544]
skeleton query  2  took 2.715517282485962 to run
starting angle calculations
[[10136.47551929 10111.82422953  -268.56398864]
 [10119.99789015 10087.35099194  -269.9840746 ]
 [10097.20051154 10060.86868301  -248.62344536]
 ...
 [20328.33377694  7082.440377    -272.04016812]
 [20332.42638649  7095.56092515  -272.5593358 ]
 [20332.92651091  7101.41031981  -331.918547  ]]
mesh_layer update
mesh_layer update


DEBUG:PYME.LMVis.visCore:Added layer, datasouce=simulated-points


pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
pointcloud.update_from_datasource() - dsname=simulated-points
angle calculations  2  took 81.60994362831116 to run
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
Viewport:  (982, 1254)
uint8 (982, 1254, 3) 255
Loop # 2 took 532.3011083602905 to run


C:\Users\laf62\.conda\envs\pyme\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Input datasources:  ['_Localizations', 'FitResults', 'surf0', 'skeleton0', 'nn-mean-skel-10nm', 'simulated-points']
Creating layers GUI
hl update
hl update
hl update
hl update
height:  97
NB best size: wx.Size(3, 88)


In [ ]:
poop = [[3,2,1],[1,2,3]]
np.shape(poop)

In [ ]:
skel_vecs = rtn_output['vecs_hat_ind'][0]
loc_vecs = rtn_output['v_hat'][0]
sim_vecs = rtn_output['sim_v'][0]
sim_skel_vecs = rtn_output['vecs_hat_sim'][0]
hand = np.cross(skel_vecs,loc_vecs)
sim_hand = np.cross(sim_skel_vecs, sim_vecs)
zhand = hand[:,2]
sim_zhand = sim_hand[:,2]
hand_bool = np.ones(len(zhand))
for neg in range(len(zhand)):
    if zhand[neg] >= 0:
        continue
    elif zhand[neg] < 0:
        hand_bool[neg] = hand_bool[neg] * -1

sim_hand_bool = np.ones(len(sim_zhand))
for neg_sim in range(len(sim_zhand)):
    if sim_zhand[neg_sim] >= 0:
        continue
    elif sim_zhand[neg_sim] < 0:
        sim_hand_bool[neg_sim] = sim_hand_bool[neg_sim] * -1

### Creating an average cross-section
import math
import matplotlib.pyplot as plt



theta = rtn_output['angles'][0]
phi = rtn_output['sim_angles'][0]
#theta = theta + (np.pi/2)
theta = theta*hand_bool
phi = phi*sim_hand_bool
r = rtn_output['skel_dists'][0]
r_sim = rtn_output['sim_dists'][0]
#theta = theta * (180/np.pi)
x = r*np.cos(theta)
y = r*np.sin(theta)
z = np.zeros(len(x))
x_sim = r_sim*np.cos(phi)
y_sim = r_sim*np.sin(phi)
#add_ds_from_Nx3(avg_cs, pipeline, pymevis, ds_name='average-cross-section', normals=None)
plt.scatter(-y,x,alpha=0.01,s=50)
plt.scatter(-y_sim,x_sim,alpha=0.01,s=50)
plt.ylim([-100,100])
plt.xlim([-100,100])

In [ ]:
#min(rtn_output['angles'][0])
#max(theta)
theta

In [ ]:
theta[1]

In [ ]:
from numpy import pi
ps = rtn_output['ps_angle']
ps_deg = ps * (180/pi)
ps_low = ps_deg > 85 # filter out angles below 85 degrees
ps_high = ps_deg < 95 # filter out angles above 95 degrees
ps_filt = ps_low * ps_high # combine filters
ps_deg = ps_deg[ps_filt] # apply filter
plt.hist(ps)

In [ ]:
##### Testing new way of simulating points around a tubule using the skeleton on real data
pipeline.selectDataSource('skeleton0')
d_skelly = pipeline.selectedDataSource
orig_skeleton = d_skelly._vertices['position'][d_skelly._vertices['halfedge']!=-1]

In [ ]:
import numpy as np
rng = np.random.default_rng(12345)

skeleton = rtn_output['skeleton'][0]
# skeleton = orig_skeleton
# equations for making the vectors
r = 1 # radius of points sampled in the circle on the plane
v_num = 1 # number of vectors desired (evenly spaced around the circle)
# rho = rng.random()*(2*np.pi)#
# rho = np.linspace(0, 2*np.pi, v_num)
# x = np.cos(rho) * r
# y = np.sin(rho) * r
# z = np.zeros(rho.shape)

f_all = []
for bd in range(len(skeleton)):
    p1 = skeleton[bd]
    n = rtn_output['tube_axis'][0][bd] # vector normal to the plane
    #print(n)
    #pp = arot_vecs[bd] #vector orthogonal to normal
    nabs = np.absolute(n)
    indices = (-n).argsort()[:3]
    v = np.zeros(3)
    v[indices[1]] = -n[indices[0]]
    v[indices[0]] = n[indices[1]]
    v[indices[2]] = 0
    u = np.cross(n,v)
    #normalize vectors
    v_norm = v/np.linalg.norm(v)
    u_norm = u/np.linalg.norm(u)
    f = []
    for be in range(v_num):
        rho = rng.random()*(2*np.pi)
        circ_p = p1 + r * v_norm * np.cos(rho) + r * u_norm * np.sin(rho)
        circ_vec = p1-circ_p
        f.append(skeleton[bd] + np.random.randint(50,100)*(circ_vec/np.linalg.norm(circ_vec)))
    f = np.asarray(f, dtype=np.float32)
    f_all.append(f)
f_all = np.asarray(f_all, dtype=np.float32)
f_flat = [element for sublist in f_all for element in sublist]
f_flat = np.asarray(f_flat)
# f_flat = f_flat * 100
# for bf in range(len(f_flat)):
#     for bg in range(v_num):
#         f_flat[bf] = orig_skeleton

In [ ]:
pymevis.glCanvas.layers[6].visible=True
pymevis.glCanvas.layers[6].point_size=20
pymevis.glCanvas.layers[6].method='spheres'
pymevis.glCanvas.layers[6].cmap='SolidGreen'
save_snapshot(pymevis.glCanvas, 'E:\\Skeleton2_testing\\analysis\\screenshots_v2\\rtn4screenshot_ROI' + str(rtn_roi_start))

In [ ]:
pymevis.glCanvas.layers[5].visible=True
pymevis.glCanvas.layers[5].point_size=15
pymevis.glCanvas.layers[5].method='spheres'
pymevis.glCanvas.layers[5].cmap='SolidYellow'
#save_snapshot(pymevis.glCanvas, 'E:\\Skeleton2_testing\\analysis\\screenshots_v2\\rtn4screenshot_ROI' + str(rtn_roi_start))
#save_snapshot(pymevis.glCanvas, 'E:\\Skeleton2_testing\\analysis\\screenshots_v2\\sec61Bscreenshot_ROI' + str(sec_roi_start))
save_snapshot(pymevis.glCanvas, 'E:\\Skeleton2_testing\\analysis\\screenshots_v2\\kdelscreenshot_ROI' + str(kdel_roi_start))

In [7]:
############################## Sec61B Analysis ##############################
sec_output = point_angles_V3(file_list=sec_list, mesh_list=sec_mesh_list, skel_list=sec_skel_list, 
                             param_list=sec_param, sim_p = sim_p) 

start
Creating Pipeline


DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.localisations.Pipelineify object at 0x000001537A8F60A8>
DEBUG:PYME.recipes.localisations:no events found
DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.localisations.ProcessColour object at 0x000001556BE31048>
DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.tablefilters.FilterTable object at 0x000001556AADCCA8>
DEBUG:PYME.recipes.recipe:Module timings: {'localisations.AddPipelineDerivedVars': 0.0009703636169433594, 'localisations.ProcessColour': 0.008002758026123047, 'tablefilters.FilterTable': 0.006984710693359375, 'localisations.ExtractTableChannel': 0.0, 'surface_fitting.DistanceToMesh': 3.780163288116455}


recipe.execute()
Pipeline Created


DEBUG:PYME.LMVis.VisGUI:Creating tabs
DEBUG:PYME.LMVis.VisGUI:Finished creating tabs


0
Gui stuff done
loading mesh
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (29426, 3), faces.shape = (59836, 3)
iterating edges
done initializing halfedges
mesh_layer update
mesh_layer update
mesh_layer update


...

pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
mesh_layer update
mesh_layer update
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
hl update
mesh_layer update
mesh_layer update
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
[[ 6544.211    13648.729      -77.736305]
 [ 7976.2188    5581.324     -127.26654 ]
 [ 9417.691     9675.6875     -68.72522 ]
 ...
 [10083.415    15761.046     -165.95262 ]
 [ 5887.503     9190.902      -86.23882 ]
 [10081.526    15767.206     -160.7525  ]]
starting skeleton query
search here
[10319 33806 65725 ... 83330  3606 83330]
[ 22.44976456 199.83739553 124.90206147 ...  35.38747623  78.35006196
  36.53317039]
skeleton query  2  took 1.0709502696990967 to run
starting angle calculations
[[ 5640.9270753   8201.47133436   -89.75017205]
 [ 5647.29426533  8193.30155962  -211.3949617 ]
 [ 5551.07515298  8191.13593332   -52.15738   ]
 ...
 [12874.1196803  17312.46795532   -70.23194255]
 [12842.70300134 17306.48442216  

DEBUG:PYME.LMVis.visCore:Added layer, datasouce=simulated-points


pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
pointcloud.update_from_datasource() - dsname=simulated-points
angle calculations  2  took 39.537129163742065 to run
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
Viewport:  (662, 1054)
uint8 (662, 1054, 3) 255
Loop # 2 took 449.06966042518616 to run


C:\Users\laf62\.conda\envs\pyme\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Disposing of FilterEditor
<PYME.ui.filterPane.FilterPanel object at 0x000001556F1511F8> <PYME.ui.filterPane.FilterPanel object at 0x000001556F1511F8>
<wx._core.ListCtrl object at 0x000001556F151318> <wx._core.ListCtrl object at 0x000001556F151318>
Disposing of CBEditor
Disposing of FilterEditor
<PYME.ui.filterPane.FilterPanel object at 0x00000155668F69D8> <PYME.ui.filterPane.FilterPanel object at 0x00000155668F69D8>
<wx._core.ListCtrl object at 0x00000155668F6798> <wx._core.ListCtrl object at 0x00000155668F6798>
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Input datasources:  ['_Localizations', 'FitResults', 'surf0', 'skeleton0', 'nn-mean-skel-10nm', 'simulated-points']
Creating layers GUI
hl update
hl update
hl update
hl update
height:  97
NB best size: wx.Size(3, 88)


In [8]:
############################## KDEL Analysis ##############################
kdel_output = point_angles_V3(file_list=kdel_list, mesh_list=kdel_mesh_list, skel_list=kdel_skel_list, 
                              param_list=kdel_param, sim_p = sim_p) 

start
Creating Pipeline


DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.localisations.Pipelineify object at 0x000001537A8E46A8>
DEBUG:PYME.recipes.localisations:no events found
DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.localisations.ProcessColour object at 0x000001563D8896A8>
DEBUG:PYME.recipes.recipe:Executing <PYME.recipes.tablefilters.FilterTable object at 0x000001555C6EA348>
DEBUG:PYME.recipes.recipe:Module timings: {'localisations.AddPipelineDerivedVars': 0.0, 'localisations.ProcessColour': 0.01096963882446289, 'tablefilters.FilterTable': 0.01109623908996582, 'localisations.ExtractTableChannel': 0.0, 'surface_fitting.DistanceToMesh': 1.8605139255523682}
ERROR:PYME.ui.layerFoldPanel:Error refreshing layer caption
Traceback (most recent call last):
  File "c:\users\laf62\code\python-microscopy\PYME\ui\layerFoldPanel.py", line 122, in _refresh
    self.Refresh()
RuntimeError: wrapped C/C++ object of type LayerCaptionBar has been deleted
ERROR:PYME.ui.layerFoldPanel:Error refreshing layer caption
T

recipe.execute()
Pipeline Created


DEBUG:PYME.LMVis.VisGUI:Creating tabs
DEBUG:PYME.LMVis.VisGUI:Finished creating tabs


0
Gui stuff done
loading mesh
Data munged to vertices, faces
initializing halfedges ...
vertices.shape = (42752, 3), faces.shape = (86304, 3)
iterating edges


...

pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
recipe.execute()
mesh_layer update
mesh_layer update
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
hl update
mesh_layer update
mesh_layer update


DEBUG:PYME.recipes.recipe:Module timings: {'localisations.AddPipelineDerivedVars': 0.0010035037994384766, 'localisations.ProcessColour': 0.014980077743530273, 'tablefilters.FilterTable': 0.031243324279785156, 'localisations.ExtractTableChannel': 0.0, 'surface_fitting.DistanceToMesh': 2.2242488861083984}


pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
mesh_layer update
mesh_layer update
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
hl update
mesh_layer update
mesh_layer update
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
[[11521.94      9508.314       67.36998 ]
 [13857.089    14340.538      138.88638 ]
 [16011.053    14344.698       25.392326]
 ...
 [15535.238     9495.116       67.80661 ]
 [10442.356    16617.682      271.33246 ]
 [10452.341    16645.445      262.3123  ]]
starting skeleton query
search here
[ 30320  72614 109552 ... 102532   9848   9848]
[ 93.65885047 118.38449695 226.90329631 ... 104.38345322 267.54380765
 266.47620751]
skeleton query  2  took 1.5476317405700684 to run
starting angle calculations
[[ 9615.0296092   4314.46512618   169.40419848]
 [ 9561.51060117  4287.59334622    88.28793072]
 [ 9606.81935042  4321.71864278   156.25328216]
 ...
 [15876.16707119 13082.60211693    93.09639856]
 [15989.71915142 13084.0675

DEBUG:PYME.LMVis.visCore:Added layer, datasouce=simulated-points


pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
pointcloud.update_from_datasource() - dsname=simulated-points
angle calculations  2  took 51.1441216468811 to run
pointcloud.update_from_datasource() - dsname=nn-mean-skel-10nm
Viewport:  (982, 894)
uint8 (982, 894, 3) 255
Loop # 2 took 513.6632475852966 to run
Disposing of FilterEditor
<PYME.ui.filterPane.FilterPanel object at 0x000001555C6EF288> <PYME.ui.filterPane.FilterPanel object at 0x000001555C6EF288>
<wx._core.ListCtrl object at 0x000001555C6EF3A8> <wx._core.ListCtrl object at 0x000001555C6EF3A8>
Disposing of CBEditor
Disposing of FilterEditor
<PYME.ui.filterPane.FilterPanel object at 0x000001557428FDC8> <PYME.ui.filterPane.FilterPanel object at 0x000001557428FDC8>
<wx._core.ListCtrl object at 0x000001557428FB88> <wx._core.ListCtrl object at 0x000001557428FB88>
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Disposing of HistLimitsEditor
Input datasources:  ['_Localizations'

In [ ]:
skel_vecs = kdel_output['vecs_hat_ind'][0]
loc_vecs = kdel_output['v_hat'][0]
sim_vecs = kdel_output['sim_v'][0]
sim_skel_vecs = kdel_output['vecs_hat_sim'][0]
hand = np.cross(skel_vecs,loc_vecs)
sim_hand = np.cross(sim_skel_vecs, sim_vecs)
zhand = hand[:,2]
sim_zhand = sim_hand[:,2]
hand_bool = np.ones(len(zhand))
for neg in range(len(zhand)):
    if zhand[neg] >= 0:
        continue
    elif zhand[neg] < 0:
        hand_bool[neg] = hand_bool[neg] * -1

sim_hand_bool = np.ones(len(sim_zhand))
for neg_sim in range(len(sim_zhand)):
    if sim_zhand[neg_sim] >= 0:
        continue
    elif sim_zhand[neg_sim] < 0:
        sim_hand_bool[neg_sim] = sim_hand_bool[neg_sim] * -1

### Creating an average cross-section
import math
import matplotlib.pyplot as plt



theta = kdel_output['angles'][0]
phi = kdel_output['sim_angles'][0]
#theta = theta + (np.pi/2)
theta = theta*hand_bool
phi = phi*sim_hand_bool
r = kdel_output['skel_dists'][0]
r_sim = kdel_output['sim_dists'][0]
#theta = theta * (180/np.pi)
x = r*np.cos(theta)
y = r*np.sin(theta)
z = np.zeros(len(x))
x_sim = r_sim*np.cos(phi)
y_sim = r_sim*np.sin(phi)
#add_ds_from_Nx3(avg_cs, pipeline, pymevis, ds_name='average-cross-section', normals=None)
fig = plt.figure(figsize=[8,8])
plt.scatter(-y[20000:30000],x[20000:30000],alpha=0.1,s=50)
#plt.scatter(-y_sim[20000:30000],x_sim[20000:30000],alpha=0.01,s=50)
plt.ylim([-200,200])
plt.xlim([-200,200])
plt.xlabel('Simulated_Data')

In [ ]:
### Creating an average cross-section
import math
import matplotlib.pyplot as plt
from numpy import pi

phi = kdel_output['angles'][0]
phi_mirror = -phi + 2*pi
#phi_mirror = np.flip(phi_mirror)
theta = np.concatenate((phi,phi_mirror))

r = np.concatenate((kdel_output['skel_dists'][0],kdel_output['skel_dists'][0]))
x = r*np.cos(theta)
y = r*np.sin(theta)
z = np.zeros(len(x))
avg_cs = np.c_[x.ravel(),y.ravel(),z.ravel()]
#add_ds_from_Nx3(avg_cs, pipeline, pymevis, ds_name='average-cross-section', normals=None)
plt.scatter(x,y)
plt.xlim([-1000,1000])
plt.ylim([-1000,1000])

In [ ]:
pymevis.glCanvas.layers[6].visible=True
pymevis.glCanvas.layers[6].point_size=15
pymevis.glCanvas.layers[6].method='spheres'

In [ ]:
# Plots a histogram for each set of data onto one graph
import matplotlib.pyplot as plt
from numpy import pi
plt.close(fig=None)

# Initialize lists
n_sim = []
bins_sim = []
patches_sim = []
bin_num = 20

# for j in loops:
    # Plot histogram
#     if j > 0:
#         n, bins, patches = plt.hist(x=sim_output['angles'][int(j)],bins=bin_num, alpha=0.2, rwidth=0.85, 
#                                     density=True, color='tab:grey', range=(0,pi))
#     else:
#         n, bins, patches = plt.hist(x=sim_output['angles'][int(j)],bins=bin_num, alpha=0.05, label = 'Simulated', 
#                                     rwidth=0.85, density=True, color='tab:grey', range=(0,pi))

#     # Append plot variables to list
#     n_sim.append([n])
#     bins_sim.append([bins])
#     patches_sim.append([patches])
#     # delete variables that are unique to each loop
#     del n, bins, patches

n_rtn = []
n_rtn_sim = []
bins_rtn = []
bins_rtn_sim = []
patches_rtn = []
patches_rtn_sim = []
for k in range(len(rtn_output['angles'])):
    # Plot histogram
    if k > 0:
        n, bins, patches = plt.hist(x=rtn_output['angles'][int(k)],bins=bin_num, alpha=0.2, 
                                    rwidth=0.85, density=True, color='lime', range=(0,pi))
        sim_n, sim_bins, sim_patches = plt.hist(x=rtn_output['sim_angles'][int(k)],bins=bin_num, alpha=0.2, 
                                    rwidth=0.85, density=True, color='grey', range=(0,pi))
    else:
        n, bins, patches = plt.hist(x=rtn_output['angles'][int(k)],bins=bin_num, alpha=0.2, label = 'Rtn4', 
                                    rwidth=0.85, density=True, color='lime', range=(0,pi))
        sim_n, sim_bins, sim_patches = plt.hist(x=rtn_output['sim_angles'][int(k)],bins=bin_num, alpha=0.2, 
                                    label = 'Rtn4_sim', rwidth=0.85, density=True, color='grey', range=(0,pi))

    # Append plot variables to list
    n_rtn.append([n])
    bins_rtn.append([bins])
    patches_rtn.append([patches])
    # delete variables that are unique to each loop
    del n, bins, patches
    
# n_sec = []
# bins_sec = []
# patches_sec = []
# for l in range(len(sec_output['angles'])):
#     # Plot histogram
#     if l > 0:
#         n, bins, patches = plt.hist(x=sec_output['angles'][int(l)],bins=bin_num, alpha=0.05, rwidth=0.85, density=True, color='magenta', range=(0,pi))
#     else:
#         n, bins, patches = plt.hist(x=sec_output['angles'][int(l)],bins=bin_num, alpha=0.05, label = 'Sec61B', rwidth=0.85, density=True, color='magenta', range=(0,pi))

#     # Append plot variables to list
#     n_sec.append([n])
#     bins_sec.append([bins])
#     patches_sec.append([patches])
#     # delete variables that are unique to each loop
#     del n, bins, patches
    
# n_kdel = []
# bins_kdel = []
# patches_kdel = []
# for l in range(len(angles_kdel_filt)):
#     # Plot histogram
#     if l > 0:
#         n, bins, patches = plt.hist(x=angles_kdel_filt[int(l)],bins=bin_num, alpha=0.05, rwidth=0.85, density=True, 
#                                     color='gold', range=(0,pi))
#     else:
#         n, bins, patches = plt.hist(x=angles_kdel_filt[int(l)],bins=bin_num, alpha=0.05, label = 'KDEL', rwidth=0.85, 
#                                     density=True, color='gold', range=(0,pi))

#     # Append plot variables to list
#     n_kdel.append([n])
#     bins_kdel.append([bins])
#     patches_kdel.append([patches])
#     # delete variables that are unique to each loop
#     del n, bins, patches
    
# n_tru, bins_tru, patches_tru = plt.hist(x=angles_tru,bins=bin_num, alpha=0.2, label = 'True', rwidth=0.85, density=True, color='cyan', range=(0,pi))

# n_rtn, bins_rtn, patches_rtn = plt.hist(x=angles_rtn, bins=bin_num, alpha=0.5, rwidth=0.85, label='Rtn4', density=True, color='lime', range=(0,pi))
# n_Sec61B, bins_Sec61B, patches_Sec61B = plt.hist(x=angles_Sec61B, bins=bin_num, alpha=0.5, rwidth=0.85, label='Sec61B', density=True, color='magenta', range=(0,pi))
# plt.close(fig=None)

# Calculate average histogram


plt.xlabel('Angle')
plt.ylabel('Normalized # of events')
plt.legend()

In [ ]:
from PYME.LMVis import gl_render3D_shaders

In [ ]:
#pymevis.glCanvas.recenter
pymevis.OnResetRotation()
pymevis.SetFit()
pymevis.RefreshView()

In [ ]:
##### ARCHIVED CODE FOR REFERENCE ONLY
##### This section averages the skeleton and creates vectors that point to a nearest neighbor at least x distance away
# Obtain vertex positions of the skeleton and make KDTree from it
pipeline.selectDataSource('skeleton0')
skelly = pipeline.selectedDataSource
skeleton = skelly._vertices['position'][skelly._vertices['halfedge']!=-1]
from scipy.spatial import cKDTree
tree_skeleton = cKDTree(skeleton)

# Average the locations of the skeleton to reduce the Q-tip effect
a_ind = tree_skeleton.query_ball_point(skeleton, 20)
skeleton[a_ind[0]]

mean_skel = np.zeros([len(skeleton),3])
for ba in range(len(skeleton)):
    mean_skel[ba] = np.mean(skeleton[a_ind[ba]],axis=0)

# Make KDTree based on the mean skeleton
mean_tree_skeleton = cKDTree(mean_skel)

# Find the closest neighbor that is at least 'min_dist' away from point being queried.
nneigh_ind = np.zeros([len(mean_skel),1])
nneigh_dist = np.zeros([len(mean_skel),1])
min_dist = 10
for aa in range(len(skeleton)):
    dist, ind = mean_tree_skeleton.query(mean_skel[aa], k=len(mean_skel))#, distance_upper_bound=1000)
    for ab in range(len(dist)):
        if dist[ab] > min_dist:
            nneigh_ind[aa] = ind[ab]
            nneigh_dist[aa] = dist[ab]
            break

# Calculate vectors pointing from each point to its nearest neighbor that is at least 'min_dist' away.
skel_vecs = np.zeros([len(mean_skel),3])
for ac in range(len(skeleton)):
    vec = mean_skel[ac] - mean_skel[int(nneigh_ind[ac][0])]
    skel_vecs[int(ac)] = vec#np.append([skel_vecs], [vec], axis=0)

# Normalize
skel_dist_mag = np.c_[nneigh_dist.ravel(),nneigh_dist.ravel(),nneigh_dist.ravel()]
vecs_hat = skel_vecs/skel_dist_mag

# Add those vectors to the skeleton in the GUI
add_ds_from_Nx3(mean_skel, pipeline, pymevis, ds_name=f'nn-mean-skel-{min_dist}nm', normals=vecs_hat)
##### The skeleton averaging and vector creation section ends here